In [1]:
!sudo apt-get update
!sudo apt-get install flex bison

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,009 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,669 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal In

In [401]:
%%writefile lexer.l
%{
#include "parser.tab.h"
%}



%%


[ \t]   ;    /* skip whitespace */
"print"   { return PRINT; }
"true"   { return TRUE; }
"false"   { return FALSE; }
"if"   { return IF; }
"else"   { return ELSE; }
"for"   { return FOR; }
"AND"   { return AND; }
"OR"   { return OR; }
"NOT"   { return NOT; }

"<"     { return '<'; }
">"     { return '>'; }
"<="    { return LE; }
">="    { return GE; }
"=="    { return EQ; }
"!="    { return NEQ; }

\".*\"    { yylval.strval = yytext; return TEXT; }
[a-zA-Z]+ { yylval = *yytext - 'a'; return ID; }
[0-9]+  { yylval = atoi(yytext); return INTEGER; }
[\n]    { return EOL; }
.       { return yytext[0]; }


%%

int yywrap(void) {
    return 1;
}

Overwriting lexer.l


In [409]:
%%writefile parser.y
%{
#include <stdio.h>
#include <stdlib.h>
extern int yylex(void);
extern char *yytext;
void yyerror(char *);
int variables[26];
%}




%token INTEGER
%token STRING
%token TEXT
%token EOL
%token ID
%token PRINT
%token BOOLEAN
%token TRUE FALSE
%left IF ELSE FOR
%left AND OR
%nonassoc NOT
%left '+' '-'
%left '*' '/'
%nonassoc '='
%nonassoc '<' '>' 
%nonassoc EQ NEQ LE GE

%%

program:
    /* empty */
    | program line
    ;

line:
    EOL
    | expr EOL { printf("= %d\n", $1); }
    ;

expr:
    INTEGER 
    | ID              { $$ = variables[$1];  }
    | ID '=' expr     { variables[$1] = $3; $$ = $3; }
    | '(' expr ')'    { $$ = $2; }
    | expr '+' expr   { $$ = $1 + $3; }
    | expr '-' expr   { $$ = $1 - $3; }
    | expr '*' expr   { $$ = $1 * $3; }
    | expr '/' expr   { $$ = $1 / $3; }
    STRING
    | 'p' '(' expr ')' { printf("%d\n", $3); }      
    | TRUE           { $$ = 1; }
    | FALSE          { $$ = 0; }
    | expr AND expr  { $$ = ($1 && $3); }
    | expr OR expr   { $$ = ($1 || $3); }
    | NOT expr       { $$ = !$2; }
    | expr '<' expr  { $$ = ($1 < $3); }
    | expr '>' expr  { $$ = ($1 > $3); }
    | expr EQ expr   { $$ = ($1 == $3); }
    | expr NEQ expr  { $$ = ($1 != $3); }
    | expr LE expr   { $$ = ($1 <= $3); }
    | expr GE expr   { $$ = ($1 >= $3); }
    | IF '(' expr ')' expr ELSE expr 
        { $$ = $3 ? $5 : $7; }
    
    ;
%%

int main(void) {
    yyparse();
    return 0;
}

void yyerror(char *s) {
    fprintf(stderr, "error: %s\n", yytext);
}


Overwriting parser.y


In [410]:
!flex lexer.l && bison -d parser.y

In [411]:
!gcc -o parser lex.yy.c parser.tab.c -lfl

lexer.l: In function ‘yylex’:
lexer.l:28:9: error: request for member ‘strval’ in something not a structure or union
   28 | \".*\"    { yylval.strval = yytext; return TEXT; }
      |         ^


In [422]:
%%writefile expresion.txt
b = 16


Overwriting expresion.txt


In [423]:
!./parser <  expresion.txt

= 16


In [424]:
%%writefile ejemplo.txt
true AND true


Overwriting ejemplo.txt


In [425]:
!./parser <  ejemplo.txt

= 1


In [426]:
%%writefile ejemplo2.txt
a=5
b=4

if (a < b) 5 else 4


Overwriting ejemplo2.txt


In [427]:
!./parser <  ejemplo2.txt

= 5
= 4
= 4


In [428]:
%%writefile ejemplo3.txt

a=4
b=4

a != b

Overwriting ejemplo3.txt


In [429]:
!./parser <  ejemplo3.txt

= 4
= 4
= 0
